In [236]:
import unittest

import hypothesis.strategies as st
from hypothesis import given, settings, note

import matplotlib.pyplot as plt
import numpy as np
import math
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import CCXGate, CXGate, CSwapGate, HGate, SwapGate, CPhaseGate
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction
from math import gcd # greatest common divisor

In [237]:
"""Function to compute the elements of Z_n."""
def multiplicative_group(n):
    """Returns the multiplicative group modulo n.

    Args:
        n: Modulus of the multiplicative group.
    """
    print("multiplicative group")
    assert n > 1
    group = [1]
    for x in range(2, n):
        if math.gcd(x, n) == 1:
            group.append(x)
    return group

In [248]:
def general_modular_exp(a, N, power):
    mult_group = multiplicative_group(N)
    if a not in mult_group:
        raise ValueError("'a' must be in " + str(mult_group))
    print(mult_group)

general_modular_exp(7, 15, 2)

multiplicative group
[1, 2, 4, 7, 8, 11, 13, 14]


In [249]:
def VBE_Adder(qubit_size):
    #outputs a + b
    #inverse outputs b - a (but in two's complement)
    #move to carry
    qc = QuantumCircuit(3*qubit_size + 1)
    for i in range(qubit_size-1):
        qc.ccx(i, i+qubit_size, 2+i+(2*qubit_size))
    
    #toffoli to second register
    qc.ccx(qubit_size-1, (2*qubit_size)-1, 2*qubit_size)
    
    #add a to b
    for i in range(qubit_size):
        qc.cx(i, i+qubit_size)
    
    #second register carry and last b
    for i in range(qubit_size-1):
        qc.ccx(i+qubit_size, 1+i+(2*qubit_size), 2+i+(2*qubit_size))
        
    qc.ccx((2*qubit_size)-1, 3*qubit_size, 2*qubit_size)
    
    qc.cx(3*qubit_size, (2*qubit_size)-1)
    
    #adder overflow
    for i in range(qubit_size-1):
        qc.ccx((2*qubit_size) - 2 - i, (3*qubit_size) - i - 1, (3*qubit_size) - i) 
        qc.cx((qubit_size) - 2 - i, (2*qubit_size) - 2 - i)
        qc.ccx((qubit_size) - 2 - i, (2*qubit_size) - 2 - i, (3*qubit_size) - i) 
        qc.cx((qubit_size) - 2 - i, (2*qubit_size) - 2 - i) 
        qc.cx((3*qubit_size) - i - 1, (2*qubit_size) - 2 - i) 
    
    #print(qc.draw(output='text'))
    qc.name = "VBE ADDER"
    #qc.to_gate()
    return qc    

In [252]:
def Modular_adder(qubit_size, N):
    binN = bin(N)[2:].zfill(qubit_size)[::-1]
        
    #move to carry
    qc = QuantumCircuit(4*qubit_size + 2)
    
    qc.append(VBE_Adder(size), [i for i in range(3*size+1)])
    
    for i in range(qubit_size):
        qc.swap(i, i + 1 + 3*qubit_size)
    
    qc.append(VBE_Adder(size).inverse(), [i for i in range(3*size+1)])
    
    qc.x(2*qubit_size)
    qc.cx(2*qubit_size, 4*qubit_size+1)
    qc.x(2*qubit_size)
    
    for i in range(qubit_size):
        if binN[i] == "1":
            qc.cx(4*qubit_size + 1, i) 
    
    qc.append(VBE_Adder(size), [i for i in range(3*size+1)])
    
    for i in range(qubit_size):
        if binN[i] == "1":
            qc.cx(4*qubit_size + 1, i) 
    
    for i in range(qubit_size):
        qc.swap(i, i + 1 + 3*qubit_size)
    
    qc.append(VBE_Adder(size).inverse(), [i for i in range(3*size+1)])
    
    qc.cx(2*qubit_size, 4*qubit_size+1)
    
    qc.append(VBE_Adder(size), [i for i in range(3*size+1)])
    
    #for i in range(qubit_size):
        
    
    
    print(qc.draw(output='text'))
    #qc.to_gate()
    return qc  

Modular_adder(4,11)

      ┌─────────────┐            ┌────────────────┐               ┌───┐     »
 q_0: ┤0            ├─X──────────┤0               ├───────────────┤ X ├─────»
      │             │ │          │                │               └─┬─┘┌───┐»
 q_1: ┤1            ├─┼──X───────┤1               ├─────────────────┼──┤ X ├»
      │             │ │  │       │                │                 │  └─┬─┘»
 q_2: ┤2            ├─┼──┼──X────┤2               ├─────────────────┼────┼──»
      │             │ │  │  │    │                │                 │    │  »
 q_3: ┤3            ├─┼──┼──┼──X─┤3               ├─────────────────┼────┼──»
      │             │ │  │  │  │ │                │                 │    │  »
 q_4: ┤4            ├─┼──┼──┼──┼─┤4               ├─────────────────┼────┼──»
      │             │ │  │  │  │ │                │                 │    │  »
 q_5: ┤5            ├─┼──┼──┼──┼─┤5               ├─────────────────┼────┼──»
      │             │ │  │  │  │ │                │             

In [266]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

size = 4
N = 7

qc = QuantumCircuit(4*size + 2)

# a = 3,2,1,0
qc.x(0)
qc.x(1)
qc.x(2)
#qc.x(3)

# b = 8,7,6,5,4
qc.x(4)
qc.x(5)
qc.x(6)
#qc.x(7)

# carry = 12,11,10,9

# modulus N = 16,15,14,13
binN = bin(N)[2:].zfill(size)[::-1]
for i in range(size):
    if binN[i] == "1":
        qc.x(3*size + 1 + i) 

# temporary carry = 17 

print(qc)

qc.measure_all()

#qc.append(VBE_Adder(size), [i for i in range(3*size+1)])
qc.append(Modular_adder(size,N), [i for i in range(4*size+2)])
qc.measure_all()

backend = Aer.get_backend('aer_simulator') 
job = execute(qc, backend, shots=1, memory=True)
readings = job.result().get_memory()
second_set = readings[0][0:4*size+2]
first_set = readings[0][(4*size)+3:(8*size)+6]
print(readings)
print(first_set)
print(second_set)

print('a     =  ' + str(first_set[3*size + 2: 4*size + 2]))
print('b     = ' + str(first_set[2*size + 1: 3*size + 2]))
print('b out = ' + str(second_set[2*size + 1: 3*size + 2]))
#print('carry =  ' + str(second_set[0:size]))


      ┌───┐
 q_0: ┤ X ├
      ├───┤
 q_1: ┤ X ├
      ├───┤
 q_2: ┤ X ├
      └───┘
 q_3: ─────
      ┌───┐
 q_4: ┤ X ├
      ├───┤
 q_5: ┤ X ├
      ├───┤
 q_6: ┤ X ├
      └───┘
 q_7: ─────
           
 q_8: ─────
           
 q_9: ─────
           
q_10: ─────
           
q_11: ─────
           
q_12: ─────
      ┌───┐
q_13: ┤ X ├
      ├───┤
q_14: ┤ X ├
      ├───┤
q_15: ┤ X ├
      └───┘
q_16: ─────
           
q_17: ─────
           
      ┌─────────────┐            ┌────────────────┐               ┌───┐     »
 q_0: ┤0            ├─X──────────┤0               ├───────────────┤ X ├─────»
      │             │ │          │                │               └─┬─┘┌───┐»
 q_1: ┤1            ├─┼──X───────┤1               ├─────────────────┼──┤ X ├»
      │             │ │  │       │                │                 │  └─┬─┘»
 q_2: ┤2            ├─┼──┼──X────┤2               ├─────────────────┼────┼──»
      │             │ │  │  │    │                │                 │    │  »
 q_3: ┤3  